## Training classifier

Using SmolVLM for extracting text and image embeddings

Can expand to Qwen2.5VM if needed

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [2]:
!pip install qwen_vl_utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 42.4 MB/s eta 0:00:00


In [3]:
!pip install num2words

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 5.5 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=190fa59d948b9deb8137785abd91f3c54ec9ca46a5482da8f1d53ccc6ec52f0d
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
Successfully built docopt


In [4]:
!pip install scikit-multilearn

In [5]:
!pip install -q accelerate datasets peft bitsandbytes tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
bigframes 

In [6]:
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 61.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.7.4.post1-cp311-cp311-linux_x86_64.whl size=187815463 sha256=d944fc7d2f962bce83fc4708c2fc0c21eaf8255962a0b350ae919362a51b7ef2
  Stored in directory: /root/.cache/pip/wheels/3d/88/d8/284b89f56af7d5bf366b10d6b8e251ac8a7c7bf3f04203fb4f
Successfully built flash-attn


In [7]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor, AutoModelForImageTextToText, TrainingArguments, Trainer, BitsAndBytesConfig, Idefics3ForConditionalGeneration
from qwen_vl_utils import process_vision_info
from PIL import Image
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from skmultilearn.model_selection import iterative_train_test_split
from tqdm import tqdm
from collections import defaultdict
import json
import base64
import os
import re

2025-04-24 07:54:28.407144: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745481268.693528      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745481268.774743      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

NotImplementedError: Mounting drive is unsupported in this environment. Use PyDrive instead. See examples at https://colab.research.google.com/notebooks/io.ipynb#scrollTo=7taylj9wpsA2.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## SmolVLM2 Instruct Model

In [ ]:
USE_LORA = False
USE_QLORA = True

model_path = "HuggingFaceTB/SmolVLM2-2.2B-Instruct"
processor = AutoProcessor.from_pretrained(model_path)

try:
    # Attempt to use CUDA
    if torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        raise Exception("CUDA not available.")
except Exception as e:
    print("Error: CUDA not found or not available. Using CPU instead.")
    device = torch.device("cpu")

if USE_QLORA or USE_LORA:
    lora_config = LoraConfig(
        r=8,
        lora_alpha=8,
        lora_dropout=0.1,
        target_modules=['down_proj','o_proj','k_proj','q_proj','gate_proj','up_proj','v_proj'],
        use_dora=False if USE_QLORA else True,
        init_lora_weights="gaussian"
    )
    lora_config.inference_mode = False
    if USE_QLORA:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
        )

    model = Idefics3ForConditionalGeneration.from_pretrained(
        model_path,
        quantization_config=bnb_config if USE_QLORA else None,
        #_attn_implementation="flash_attention_2",
        device_map="auto"
    )
    model.add_adapter(lora_config)
    model.enable_adapters()
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, lora_config)
    print(model.get_nb_trainable_parameters())
else:
    model = Idefics3ForConditionalGeneration.from_pretrained(
        model_path,
        torch_dtype=torch.bfloat16,
        #_attn_implementation="flash_attention_2",
    ).to(device)

    # if you'd like to only fine-tune LLM
    for param in model.model.vision_model.parameters():
        param.requires_grad = False

## Data Loader

In [ ]:
# For example, a flattened version of your label tree:
ALL_CLASSES = [
    # Main classes
    "Abandoned Trolleys", "Animals", "Cleanliness", "Construction Sites",
    "Drains & Sewers", "Drinking Water", "Housing", "Illegal Parking",
    "Others", "Parks & Greenery", "Pests", "Roads & Footprints",
    "Shared Bicycles", "Smoking",
    "Others",  # this is often used as a catch-all category
    #Sub classes
    "Cold Storage", "FairPrice", "Giant", "Ikea", "Mustafa", "Other Trolleys", "ShengSong",
    "Bird Issues", "Cat Issues", "Dead Animal", "Dog Issues", "Injured Animal", "Other Animal Issues",
    "Bulky Waste in Common Areas", "Dirty Public Areas", "High-rise Littering", "Overflowing Litter Bin",
    "Construction Noise",
    "Choked Drain or Stagnant Water", "Damaged Drain", "Flooding", "Sewage Smell", "Sewer Choke or Overflow",
    "No Water", "Water Leak", "Water Pressure", "Water Quality",
    "Common Area Maintenance", "HDB Car Park Maintenance", "Lightning Maintenance", "Playground & Fitness Facilities Maintenance",
    "HDB or URA Car Park", "Motorcycle at Void Deck", "Road",
    "Fallen Tree or Branch", "Other Parks and Greenery Issues", "Overgrown Grass", "Park Facilities Maintenance", "Park Lighting Maintenance",
    "Bee & Hornets", "Cockroaches in Food Establishment", "Mosquitoes", "Rodents in Common Areas", "Rodents in Food Establishment",
    "Covered Linkway Maintenance", "Damaged Road Signs", "Faulty Streetlight", "Footpath Maintenance", "Road Maintenance",
    "Anywheel", "HelloRide", "Other Bicycles",
    "Food Premises", "Other Public Areas", "Parks & Park Connectors"
]

label_to_idx = {label: idx for idx, label in enumerate(ALL_CLASSES)}
NUM_CLASSES = len(ALL_CLASSES)

data_root = "./gdrive/MyDrive/Huawei Data/2. Sorted"  # Adjust to your data folder path
label_vector_file = "./label_vectors.json"  # Path to your label vector JSON file

# System prompt for the model
system_prompt = (
    "You are an expert in municipal services issues. Your task is to analyze the provided input, "
    "which may include an image and a description, and categorize the issue into one or more categories "
    "from the predefined list of municipal service issue types. Additionally, assess the severity of the issue "
    "as one of the following: Low, Medium, or High.\n\n"
    "The predefined list of categories is as follows:\n"
    + "\n".join(f"- {category}" for category in ALL_CLASSES) +
    "\n\nYour response should be in the following JSON format:\n"
    "{\n"
    "    \"categories\": [categories],\n"
    "    \"severity\": severity\n"
    "}\n\n"
    "Ensure that the categories are selected from the provided list of issue types, and the severity is determined "
    "based on the details provided in the input."
)


### Think have to redo this a bit
Thought collate_fn in the finetuning example is the data loader. collate_fn which is kinda what i made here is the batching function. Data loader just returns the dataset raw data

In [ ]:
# Define a training dataset
class FixMyStreetDataset(Dataset):
    def __init__(self, data_root, label_vector_file):
        """
        data_root: Folder containing the image and JSON files.
        label_vector_file: Path to the JSON file mapping report IDs to label lists.
        """
        self.data_root = data_root

        # Load label mapping (file names without extension as keys)
        with open(label_vector_file, 'r', encoding='utf-8') as f:
            self.label_vector = json.load(f)

        self.report_ids = list(self.label_vector.keys())

    def __len__(self):
        return len(self.report_ids)

    def __getitem__(self, idx):
        report_id = self.report_ids[idx]

        # Get label vector entry for the current report ID
        label_entry = self.label_vector[report_id]

        image_path = label_entry["image_path"]

        # Load the JSON metadata
        # Get the image path and derive the JSON path
        image_path = image_path.replace("\\", "/")  # Normalize backslashes to forward slashes
        json_path = os.path.splitext(image_path)[0] + ".json"  # Replace the image extension with .json

        # Replace the old root ("data/2. Sorted") with the new data_root
        json_path = json_path.replace("data/2. Sorted", self.data_root)
        image_path = image_path.replace("data/2. Sorted", self.data_root)

        # Normalize the path
        json_path = os.path.normpath(json_path)

        with open(json_path, 'r', encoding="utf-8") as f:
            metadata = json.load(f)

        # Prepare the text content
        input_text = metadata["description"] + "\n\n"
        input_text += "Nearby location tags: " + ", ".join([f"{k}: {v}" for tag in metadata["tags"]["nearby"] for k, v in tag.items()]) + "\n\n"
        input_text += "Enclosing location tags: " + ", ".join([f"{k}: {v}" for tag in metadata["tags"]["enclosing"] for k, v in tag.items()])

        # Check if the image exists
        if image_path:
            # Load the image
            image = Image.open(image_path)
            if image.mode != "RGB":
                image = image.convert("RGB")
        else:
            # Placeholder for missing image
            image = torch.zeros((3, 224, 224), dtype=torch.uint8)

        output_text = f'{{"categories": {json.dumps(label_entry["labels"])}, "severity": {json.dumps(label_entry["severity"])}}}'


        return input_text, image, output_text


In [ ]:
image_token_id = processor.tokenizer.additional_special_tokens_ids[
            processor.tokenizer.additional_special_tokens.index("<image>")]

# Batching function
def collate_fn(data_stream):
    texts = []
    images = []

    for data in data_stream:
        user_content = [{"type": "text", "text": data[0]}, {"type": "image"}]

        messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_content},
                {"role": "assistant", "content": data[2]}
            ]

        # Process the inputs using the processor
        text = processor.apply_chat_template(
            messages,
            add_generation_prompt=False,
        )
        texts.append(text.strip())
        images.append([data[1]])

    batch = processor(text=texts, images=images, return_tensors="pt", padding=True)
    labels = batch["input_ids"].clone()
    labels[labels == processor.tokenizer.pad_token_id] = -100
    labels[labels == image_token_id] = -100
    batch["labels"] = labels

    return batch

In [ ]:
# Create dataset and dataloader
dataset = FixMyStreetDataset(data_root=data_root,
                             label_vector_file=label_vector_file)

data_loader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

In [ ]:
len(dataset)

In [ ]:
dataset[0]

In [ ]:
def labels_to_multi_hot(label_list, label_to_idx, num_classes):
    """
    Convert a list of label strings into a multi-hot tensor.

    Parameters:
      label_list (List[str]): The labels assigned to one example.
      label_to_idx (Dict[str,int]): Mapping from each label string to its index.
      num_classes (int): Total number of possible classes.

    Returns:
      torch.Tensor of shape (num_classes,) with 1s at the indices corresponding
      to label_list, 0s elsewhere.
    """
    multi_hot = torch.zeros(num_classes, dtype=torch.float32)
    for label in label_list:
        idx = label_to_idx.get(label)
        if idx is not None:
            multi_hot[idx] = 1.0
    return multi_hot


In [ ]:
all_indices = list(range(len(dataset)))

X = np.arange(len(dataset)).reshape(-1, 1)
Y = np.stack([labels_to_multi_hot(dataset.report_ids[i], label_to_idx, NUM_CLASSES).numpy()
              for i in range(len(dataset))])

X_train, Y_train, X_test, Y_test = iterative_train_test_split(X, Y, test_size=0.2)
train_idx = X_train.ravel().tolist()
test_idx  = X_test.ravel().tolist()

train_dataset = Subset(dataset, train_idx)
test_dataset  = Subset(dataset, test_idx)

In [ ]:
train_dataset[0]

In [ ]:
test_dataset[0]

## Fine-tuning

In [ ]:
model_name = model_path.split("/")[-1]

training_args = TrainingArguments(
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=50,
    learning_rate=1e-4,
    weight_decay=0.01,
    logging_steps=25,
    save_strategy="steps",
    save_steps=250,
    save_total_limit=1,
    optim="paged_adamw_8bit", # for 8-bit, keep this, else adamw_hf
    bf16=True, # underlying precision for 8bit
    output_dir=f"./{model_name}-vqav2",
    hub_model_id=f"{model_name}-vqav2",
    report_to="tensorboard",
    remove_unused_columns=False,
    gradient_checkpointing=True
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    train_dataset=dataset,
)

In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub()

## Qwen2.5VM stuff (dont touch for now)

In [ ]:
# @title inference function
def inference(image_path, prompt, sys_prompt="You are a helpful assistant.", max_new_tokens=4096, return_input=False):
    image = Image.open(image_path)
    image_local_path = "file://" + image_path
    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": [
                {"type": "text", "text": prompt},
                {"image": image_local_path},
            ]
        },
    ]
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    print("text:", text)
    # image_inputs, video_inputs = process_vision_info([messages])
    inputs = processor(text=[text], images=[image], padding=True, return_tensors="pt")
    inputs = inputs.to('cuda')

    output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens)
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, output_ids)]
    output_text = processor.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    if return_input:
        return output_text[0], inputs
    else:
        return output_text[0]

#  base 64 编码格式
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [ ]:
model_path = "Qwen/Qwen2.5-VL-7B-Instruct"
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(model_path, torch_dtype=torch.bfloat16, attn_implementation="flash_attention_2",device_map="auto")
processor = AutoProcessor.from_pretrained(model_path)

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to(model.device)

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)